In [84]:
# -*- coding: utf-8 -*-
"""
Created on Thu Apr  8 14:37:50 2021

@author: HashiriNio
"""

#from entity_get import EntityObtainer
#from candidate_scoring import candi_scoring
#import candidate_get
#import sys

input_question = '物理层和物理层中继系统有什么共同之处吗？'
'''
entity_obtainer = EntityObtainer()
entities = entity_obtainer.mention_get(input_question)
print('得到的mention为:',entities)

candidates = []
for entity in entities:
    candidates += candidate_get.GetPaths(entity)
if candidates == []:
    sys.exit('没有符合的实体')
print(candidates)
'''
candidates = [('计算机网络', '模型'), ('计算机网络', '模型', '网络结构'),('物理层', '设备', '集线器'),('物理层中继系统', '设备', '集线器')]
scorer = candi_scoring()
the_path,the_score,mark = scorer.scoring(input_question,candidates)
answer = get_answer(the_path,mark)
if the_score > 0.45:
    print('查询路径:',the_path)
    print('答案:',answer,'\n得分:',str(the_score))
else:
    print('没有合适的答案')


loaded
['计算机网络', '物理层', '物理层中继系统']
bridged_paths [('物理层', 1, 1, '物理层中继系统')]
bridged_ques ['物理层和物理层中继系统的相同点是？', '物理层和物理层中继系统的区别是？']
0.5
jls
查询路径: ('物理层', 1, 1, '物理层中继系统')
答案: 它们的设备都有集线器。 
得分: 0.58831286


In [83]:
from neo4j import GraphDatabase
driver = GraphDatabase.driver("bolt://localhost:7687",auth=("neo4j","123456"))
session = driver.session()

def GetPaths(entity):
    #entity='物理层'
    #一跳路径
    query_1 = "match (a:Entity)-[r1:hasPart]-() where a.name=$name return DISTINCT r1.name"
    result = session.run(query_1,name=entity)
    path_one = []
    for i in result:
        path_one.append((entity,i['r1.name']))
    #两跳路径
    path_two = []
    query_2 = "match (a:Entity)-[r1:hasPart]-()-[r2:hasPart]->() where a.name=$name return DISTINCT r1.name,r2.name"
    result = session.run(query_2,name=entity)
    for i in result:
        path_two.append((entity,i['r1.name'],i['r2.name']))
    #print(path_one,'\n',path_two)
    return path_one+path_two

def bridge(index,candidates):
    #得到所有需要桥接的实体
    entities = []
    for i in range(len(index)):
        if candidates[index[i]][0] not in entities:
            entities.append(candidates[index[i]][0])
    #得到桥接后的路径
    print(entities)
    paths = []
    if len(entities)<2:
        return []
    else:
        query = "match (a:Entity)-[r1:hasPart]->()<-[r2:hasPart]-(b:Entity) where a.name=$name AND b.name=$name2 return DISTINCT r1.name,r2.name"
        query2 = "match (a:Entity)<-[r1:hasPart]-()-[r2:hasPart]->(b:Entity) where a.name=$name AND b.name=$name2 return DISTINCT r1.name,r2.name"
        end = len(entities)
        for i in range(end-1):
            for l in range(i+1,end):
                result = session.run(query,name=entities[i],name2 = entities[l])
                result2 = session.run(query2,name=entities[i],name2 = entities[l])
                for k in result: 
                    if (entities[i],1,1,entities[l]) not in paths:
                        paths.append((entities[i],1,1,entities[l]))
                for k in result2:
                    if (entities[i],1,1,entities[l]) not in paths:
                        paths.append((entities[i],1,1,entities[l]))
        return paths

def get_answer(path,mark):
    if mark == 1:#相异问题
        query = "match (a:Entity)-[r1:hasPart]->(b:Entity) where a.name=$name return DISTINCT r1.name,b.name"
        resulta = session.run(query,name = path[0])
        resultb = session.run(query,name = path[3])
        ra = {}
        for i in resulta:
            ra[i['r1.name']] = [i['b.name'],0]
        for i in resultb:
            if i['r1.name'] in ra.keys():
                ra[i['r1.name']][1] = i['b.name']
        answer = ''
        for i in ra.keys():
            answer += path[0]+'的'+i+'是'+ra[i][0]+','+path[3]+'的'+i+'是'+ra[i][1]+'。'
        return answer
    if len(path) ==2:
        query = "match (a:Entity)-[r1:hasPart]-(b:Entity) where a.name=$name return DISTINCT b.name"
        result = session.run(query,name=path[0])
    elif len(path) == 3:#两条
        query = "match (a:Entity)-[r1:hasPart]-()-[r2:hasPart]->(b:Entity) where a.name=$name return DISTINCT b.name"
        result = session.run(query,name=path[0])
    else:#桥接的路径解决相同点问题
        query = "match (a:Entity)-[r1:hasPart]->(b:Entity)<-[r2:hasPart]-(c:Entity) where a.name=$name AND c.name=$name2 return DISTINCT r1.name,b.name"
        result = session.run(query,name=path[0],name2 = path[3])
        query2 = "match (a:Entity)<-[r1:hasPart]-(b:Entity)-[r2:hasPart]->(c:Entity) where a.name=$name AND c.name=$name2 return DISTINCT r1.name,b.name"
        result2 = session.run(query2,name=path[0],name2 = path[3])
        final_answer = ''
        k = ''
        for l in result2:
            k = '它们都是'+l['b.name']+'的'+l[r1.name]
        if k != '':
            k += '。'
            final_answer += k
        count = 0
        answer = '它们的'
        for i in result:
            if count == 0:
                answer += i['r1.name']+'都有'+i['b.name']
                count += 1
            else: 
                answer += ';'+i['r1.name']+'都有'+i['b.name']#如果有多个答案 
        if answer != '它们的':
            answer += '。'
            final_answer += answer
        return final_answer
    count = 0
    for i in result:
        if count == 0:
            answer = i['b.name']
            count += 1
        else: 
            answer += (';'+i['b.name'])#如果有多个答案
    return answer

In [28]:
import numpy as np
#import pandas as pd
#import pickle
from keras_bert import Tokenizer,get_custom_objects
import codecs as cs
from keras.models import load_model
#import candidate_get

class candi_scoring():
    def __init__(self):
        self.max_seq_len = 20
        #加载bert字典
        dict_path = r'D:\final_design\Final_one\bert\vocab.txt'
        token_dict = {}
        with cs.open(dict_path, 'r', 'utf8') as reader:
            for line in reader:
                token = line.strip()
                token_dict[token] = len(token_dict)
        self.tokenizer = Tokenizer(token_dict)
        #加载评分模型
        self.model = load_model(r'D:\final_design\Final_one\data\model\model_match_general5_1.h5',custom_objects=get_custom_objects())
        print('loaded')
    
    def get_tokens(self,seq1,seq2):
        #seq1为str，seq2为list
        X1_1,X1_2,X2_1,X2_2 = [] ,[],[],[]
        x1_1,x1_2 = self.tokenizer.encode(first = seq1,max_len = self.max_seq_len)
        for i in range(len(seq2)):
            #x1_1,x1_2 = self.tokenizer.encode(first = seq1[i],max_len = self.max_seq_len)
            x2_1,x2_2 = self.tokenizer.encode(first = seq2[i],max_len = self.max_seq_len)
            X1_1.append(x1_1)
            X1_2.append(x1_2)
            X2_1.append(x2_1)
            X2_2.append(x2_2)
        
        return np.array(X1_1), np.array(X1_2), np.array(X2_1), np.array(X2_2)
    
    def get_ques(self,candidates):
        #candidates = [(,),(,)]
        candidate_paths = []
        for candidate in candidates:
            if len(candidate) == 2:
                candidate_paths.append(candidate[0]+'的'+candidate[1]+'是？')
            else:
                candidate_paths.append(candidate[0]+'的'+candidate[1]+'的'+candidate[2]+'是？')
        return candidate_paths
    
    def get_bridged_ques(self,paths):
        #paths = [(x1,r1,r2,x2),()]
        bridged_q = []
        for path in paths:
            bridged_q.append(path[0]+'和'+path[3]+'的相同点是？')
            bridged_q.append(path[0]+'和'+path[3]+'的区别是？')
        return bridged_q
    
    def scoring(self,question,candidates):
        topk = 5
        candidate_ques = self.get_ques(candidates)
        in1_1,in1_2,in2_1,in2_2 = self.get_tokens(question,candidate_ques)
        score = self.model.predict([in1_1,in1_2,in2_1,in2_2])
        score1 = []
        for i in score:
            score1.append(i[0])
        temp = sorted(score1,reverse = 1)
        Index = []
        for i in range(min(topk,len(temp))):
            Index.append(score1.index(temp[i]))
        #桥接打分
        bridged_paths = bridge(Index,candidates)
        #print('bridged_paths',bridged_paths)
        if bridged_paths == []:
            b_one = 0
        else:
            bridged_ques = self.get_bridged_ques(bridged_paths)#问题数目是路径的两倍
            #print('bridged_ques',bridged_ques)
            in1_1,in1_2,in2_1,in2_2 = self.get_tokens(question,bridged_ques)
            scoreb = self.model.predict([in1_1,in1_2,in2_1,in2_2])
            score2 = []
            for i in scoreb:
                score2.append(i[0])
            tempb = sorted(score2,reverse = 1)
            bindex = score2.index(tempb[0])
            b_one = tempb[0]
        #选择得分最高的路径
        mark = 0
        if b_one>score1[Index[0]]:
            print(bindex/2)
            the_path = bridged_paths[int(bindex/2)]
            if bindex%2 == 0:
                mark = 1
            the_score = b_one
        else:
            the_path = candidates[Index[0]]
            the_score = score1[Index[0]]
        return the_path,the_score,mark

In [7]:
i = '好家伙'

In [8]:
i+=(','+'你爸爸')
i

'好家伙,你爸爸'

In [10]:
k = (1,2,3,4)
k+=5
k

TypeError: can only concatenate tuple (not "int") to tuple

In [11]:
3%2

1

In [1]:
r = {}

In [3]:
k = r['ni']

KeyError: 'ni'

In [81]:
query = "match (a:Entity)-[r1:hasPart]->(b:Entity)<-[r2:hasPart]-(c:Entity) where a.name=$name AND c.name=$name2 return DISTINCT r1.name,b.name"
result = session.run(query,name='物理层',name2 = '物理层中继系统')

In [67]:
k=list(result)
print(result)

In [68]:
k

[<Record r1.name='设备' b.name='集线器'>]

In [63]:
final_answer = ''

In [64]:
if list(result) != []:
    print(result)
    count = 0
    answer = '它们的'
    for i in result:
        if count == 0:
            print('jls')
            answer += i['r1.name']+'都有'+i['b.name']
            count += 1
        else: 
            answer += ';'+i['r1.name']+'都有'+i['b.name']#如果有多个答案 
    answer += '。'
    final_answer += answer

In [65]:
final_answer

''

In [79]:
print(result)

In [75]:
k = result

In [82]:
len(result)

TypeError: object of type 'Result' has no len()

In [4]:
from neo4j import GraphDatabase
driver = GraphDatabase.driver("bolt://localhost:7687",auth=("neo4j","123456"))
session = driver.session()
path = ('计算机网络','描述')
query = "match (a:Entity)-[r1:hasPart]->(b:Entity) where a.name=$name AND r1.name=$name2 return DISTINCT b.name"
result = session.run(query,name=path[0],name2=path[1])
count = 0
for i in result:
    if count == 0:
        answer = i['b.name']
        count += 1
    else: 
        answer += (';'+i['b.name'])#如果有多个答案

In [5]:
answer

'计算机网络是指将地理位置不同的具有独立功能的多台计算机及其外部设备，通过通信线路连接起来，在网络操作系统，网络管理软件及网络通信协议的管理和协调下，实现资源共享和信息传递的计算机系统。...'